In [8]:
import json
import datetime
import time
from telnetlib import Telnet
import csv
dtm=datetime.datetime.now()
file=open("Output_data.csv",'w')
file.close() 


In [9]:
def printf(arg):
    timestamp=datetime.datetime.now()-dtm
    print(str(timestamp)+":      "+str(arg))

In [10]:
def blink(blinkStr):
    timestamp=datetime.datetime.now() - dtm
    with open("Output_data.csv",'a') as csvfile:
        row=[str(timestamp),blinkStr]
        csvwriter = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
        csvwriter.writerow(row)
        csvfile.close()

In [11]:
socket=13854
con= Telnet('localhost', socket)
con.write(b'{"enableRawOutput": true, "format": "Json"}')
printf("Connecting TCP Socket Host socket:{socket} ...")

0:00:02.057075:      Connecting TCP Socket Host socket:{socket} ...


In [12]:
printf(str(con)+" configured succesfully")

0:00:02.065014:      <telnetlib.Telnet object at 0x0000021644E10910> configured succesfully


In [13]:
status="NotConnected"
attention=None 
meditation = None
theta =None
delta = None
lowAlpha = None
highAlpha = None
lowBeta = None
highBeta = None
lowGamma = None
highGamma = None
__attention_records=[]

In [14]:
while True:
    line = con.read_until(b'\r');
    if len(line) > 20:
        try:
            raw_str = (str(line).rstrip("\\r'").lstrip("b'"))
            data = json.loads(raw_str)
            if "status" in data.keys():
                if status != data["status"]:
                    status = data["status"]
                    if data["status"] == "scanning":
                        printf("Scanning device..")
                    else:
                        printf("Connection lost, trying to reconnect..")
            else:
                if "eSense" in data.keys():
                    if data["eSense"]["attention"] + data["eSense"]["meditation"] == 0:
                        if status != "fitting":
                            status = "fitting"
                            printf("Fitting Device..")

                    else:
                        if status != "connected":
                            status = "connected"
                            printf("Successfully Connected ..")
                        attention = data["eSense"]["attention"]
                        meditation = data["eSense"]["meditation"]
                        theta = data['eegPower']['theta']
                        delta = data['eegPower']['delta']
                        lowAlpha = data['eegPower']['lowAlpha']
                        highAlpha = data['eegPower']['highAlpha']
                        lowBeta = data['eegPower']['lowBeta']
                        highBeta = data['eegPower']['highBeta']
                        lowGamma = data['eegPower']['lowGamma']
                        highGamma = data['eegPower']['highGamma']
                        __attention_records.append(data["eSense"]["attention"])
                        __attention_records.append(data["eSense"]["meditation"])
                        printf(data)
                elif "blinkStrength" in data.keys():
                    blinkStrength = data["blinkStrength"]
                    printf(blinkStrength)
                    blink(blinkStrength)

        except:
            printf("Read failure")


0:00:02.398768:      Scanning device..
